# NFL Game Outcome Modeling: nflfastR + RBSDM EPA

This notebook pulls **nflfastR** play-by-play using `nfl_data_py`, joins **RBSDM** team EPA/play summaries, and builds a clean game-level dataset.  It trains two models - **Logistic Regression** and **Gradient Boosting** - and compares their performance.  It also outputs **win probabilities** for a selected week.  Two spaces after periods here by your preference.  

**Data sources:**  
- nflverse / nflfastR play-by-play via `nfl_data_py`.  
- RBSDM team EPA/play CSV.  

**Sections:**  
1. Setup and installs  
2. Load data  
3. Feature engineering and lagged team-week aggregates  
4. Merge to game-level rows  
5. Train Logistic Regression and Gradient Boosting  
6. Evaluate, compare, and interpret  
7. Predict win probabilities for a given week  


## 1. Setup and installs

In [1]:
# If running locally, uncomment as needed.
%pip install -U pandas numpy scikit-learn matplotlib requests nfl_data_py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 6.8 MB/s  0:00:01m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.6 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 13.6 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.7/684.7 kB 12.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 13.9 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.3
    Uninstalling numpy-1.23.3:
      Successfully uninstalled numpy-1.23.3━━━━━━━━━━━━━━━━━━━━━━━  2/10 [numpy]
  Attempting uninstall: pandas━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/10 [joblib]
    Found existing installation: pandas 1.3.4━━━━━━━━━━━━━━━━━  3/10 [joblib]
    Uninstalling pandas-1.3.4:0m╺━━━━━━━━━━━━━━━━━━━  5/10 [pandas]
      Successfully uninstalled pandas-1.3.4━━━━━━━━━━━━━━━━━━━  5/10 [pandas]
  Attempting u

## 2. Imports and configuration

In [ ]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, brier_score_loss, accuracy_score
from sklearn.inspection import permutation_importance

# nfl_data_py imports
from nfl_data_py import import_pbp_data, import_schedules

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

# Plot defaults
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["axes.grid"] = True


## 3. Parameters

In [ ]:
# Seasons to include for training.  Adjust as desired.
SEASONS = list(range(2019, 2025))  # 2019–2024

# Week to predict (example).  Must exist in the selected seasons.
PREDICT_SEASON = 2024
PREDICT_WEEK = 10  # change to the week you want probabilities for

# RBSDM stats CSV endpoint.  If the path changes, update here.
RBSDM_URL = "https://rbsdm.com/stats/stats.csv"


## 4. Load play-by-play and schedules

In [ ]:
# Pull play-by-play for selected seasons
pbp = import_pbp_data(SEASONS)

# Keep regular plays only
pbp = pbp.loc[pbp["play_type"].notna()].copy()

# Pull schedules for outcomes and basic game info
sched = import_schedules(SEASONS)

# Outcome label: home team win as 1, else 0
sched = sched.assign(
    home_win = (sched["result"] > 0).astype(int)
)

# Keep necessary columns
sched_small = sched[[
    "game_id","season","week",
    "home_team","away_team","home_score","away_score","home_win","game_type"
]].copy()

# Filter to regular season only for modeling unless you want playoffs
sched_small = sched_small.loc[sched_small["game_type"] == "REG"].reset_index(drop=True)

sched_small.head()


## 5. Team-week aggregates and lag features

In [ ]:
# Offense-side aggregates by posteam and (season, week)
off_agg = (
    pbp.groupby(["season","week","posteam"], as_index=False)
       .agg(
           off_plays=("play_id","count"),
           off_epa_mean=("epa","mean"),
           off_success=("success","mean"),
           off_yards_gained=("yards_gained","mean"),
           off_pass_rate=("pass","mean"),
           off_rush_rate=("rush","mean")
       )
       .rename(columns={"posteam":"team"})
)

# Defense-side aggregates by defteam and (season, week)
def_agg = (
    pbp.groupby(["season","week","defteam"], as_index=False)
       .agg(
           def_plays=("play_id","count"),
           def_epa_mean=("epa","mean"),
           def_success=("success","mean"),
           def_yards_gained=("yards_gained","mean"),
           def_pass_rate=("pass","mean"),
           def_rush_rate=("rush","mean")
       )
       .rename(columns={"defteam":"team"})
)

team_week = pd.merge(off_agg, def_agg, on=["season","week","team"], how="outer")

# Add rolling (lagged) features per team up to the prior week to avoid leakage
def add_group_rolls(df, group_col="team"):
    df = df.sort_values(["season","week"]).copy()
    # For rolling across seasons, create a continuous counter by team
    df["season_week_index"] = (df["season"] - df["season"].min()) * 18 + df["week"]
    df = df.sort_values([group_col, "season_week_index"])

    num_cols = [c for c in df.columns if c not in [group_col,"season","week","season_week_index"]]

    for col in num_cols:
        # shift by 1 to exclude current week, then rolling means over last N weeks
        df[f"{col}_lag1"] = df.groupby(group_col)[col].shift(1)
        df[f"{col}_roll3"] = df.groupby(group_col)[col].shift(1).rolling(3).mean()
        df[f"{col}_roll5"] = df.groupby(group_col)[col].shift(1).rolling(5).mean()

    return df

team_week = add_group_rolls(team_week, "team")

# Keep only lagged/rolling features to avoid leakage
lag_cols = [c for c in team_week.columns if c.endswith(("_lag1","_roll3","_roll5"))]
base_cols = ["season","week","team"]
team_week_lagged = team_week[base_cols + lag_cols].copy()

team_week_lagged.head()


## 6. RBSDM team EPA/play and team-name harmonization

In [ ]:
# Download RBSDM team stats
r = requests.get(RBSDM_URL, timeout=30)
r.raise_for_status()
rbsdm = pd.read_csv(io.StringIO(r.text))

# Keep a few useful columns if available
possible_cols = [c for c in rbsdm.columns if c.lower() in {
    "team","season","off_epa","def_epa","off_success","def_success","off_pass_epa","off_rush_epa","def_pass_epa","def_rush_epa"
}]
if "team" not in possible_cols:
    possible_cols = ["team","season","off_epa","def_epa","off_success","def_success","off_pass_epa","off_rush_epa","def_pass_epa","def_rush_epa"]
rbsdm_small = rbsdm[[c for c in possible_cols if c in rbsdm.columns]].copy()

# Team mapping between abbreviations and full names for current franchises
abbr_to_full = {
    "ARI":"Arizona Cardinals","ATL":"Atlanta Falcons","BAL":"Baltimore Ravens","BUF":"Buffalo Bills",
    "CAR":"Carolina Panthers","CHI":"Chicago Bears","CIN":"Cincinnati Bengals","CLE":"Cleveland Browns",
    "DAL":"Dallas Cowboys","DEN":"Denver Broncos","DET":"Detroit Lions","GB":"Green Bay Packers",
    "HOU":"Houston Texans","IND":"Indianapolis Colts","JAX":"Jacksonville Jaguars","KC":"Kansas City Chiefs",
    "LV":"Las Vegas Raiders","LAC":"Los Angeles Chargers","LAR":"Los Angeles Rams","MIA":"Miami Dolphins",
    "MIN":"Minnesota Vikings","NE":"New England Patriots","NO":"New Orleans Saints","NYG":"New York Giants",
    "NYJ":"New York Jets","PHI":"Philadelphia Eagles","PIT":"Pittsburgh Steelers","SEA":"Seattle Seahawks",
    "SF":"San Francisco 49ers","TB":"Tampa Bay Buccaneers","TEN":"Tennessee Titans","WAS":"Washington Commanders"
}

# Create a team mapping table for joining
teams_map = pd.DataFrame({
    "team": list(abbr_to_full.keys()),
    "team_full": list(abbr_to_full.values())
})

# RBSDM uses a "team" name string.  Normalize with the full names where possible.
if "team" in rbsdm_small.columns:
    rbsdm_small = rbsdm_small.rename(columns={"team":"team_full"})
else:
    rbsdm_small["team_full"] = None  # fallback

# Join RBSDM to lagged team-week on full names
team_week_enriched = team_week_lagged.merge(teams_map, on="team", how="left").merge(
    rbsdm_small, on=["team_full","season"], how="left"
)

team_week_enriched.head()


## 7. Assemble game-level dataset

In [ ]:
# For each game, attach home team features and away team features from prior week aggregates
def join_side(team_key_prefix, team_col_name):
    side = team_week_enriched.copy()
    side = side.rename(columns={
        "team": f"{team_key_prefix}_team"
    })
    # Add prefixes to feature columns
    side_feat_cols = [c for c in side.columns if c not in ["season","week",f"{team_key_prefix}_team","team_full"]]
    side = side.rename(columns={c: f"{team_key_prefix}_{c}" for c in side_feat_cols})
    return side

home_side = join_side("home", "home_team")
away_side = join_side("away", "away_team")

games = sched_small.merge(
    home_side, left_on=["season","week","home_team"], right_on=["season","week","home_team"], how="left"
).merge(
    away_side, left_on=["season","week","away_team"], right_on=["season","week","away_team"], how="left"
)

# Target
games["y_home_win"] = games["home_win"].astype(int)

# Drop rows with no features (early weeks may lack lags)
feature_cols = [c for c in games.columns if any(c.startswith(p) for p in ["home_","away_"])]
X = games[feature_cols].copy()
y = games["y_home_win"].copy()

# Remove columns that are purely identifiers or non-numeric
X = X.select_dtypes(include=[np.number]).copy()

# Drop columns with too many missing values
min_non_na = int(0.8 * len(X))
keep_cols = [c for c in X.columns if X[c].notna().sum() >= min_non_na]
X = X[keep_cols]
games = games.assign(valid_row = X.notna().all(axis=1))
mask = games["valid_row"].values
X = X[mask]
y = y[mask]
games_valid = games[mask].reset_index(drop=True)

print(f"Total games with features: {len(games_valid)} of {len(games)}")

X.head()


## 8. Train and evaluate models

In [ ]:
# Use a season-aware split: train on seasons before the max-1, test on the last season in the set
last_season = max(SEASONS)
train_mask = games_valid["season"] < last_season
test_mask  = games_valid["season"] == last_season

X_train, y_train = X[train_mask], y[train_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

print(f"Train: {X_train.shape}, Test: {X_test.shape}  |  Train seasons <= {last_season-1}, Test season == {last_season}")

# Logistic Regression with scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

logit = LogisticRegression(max_iter=200, solver="lbfgs")
logit.fit(X_train_scaled, y_train)
logit_proba = logit.predict_proba(X_test_scaled)[:,1]

# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)
gb_proba = gb.predict_proba(X_test)[:,1]

def eval_metrics(y_true, y_pred_proba, threshold=0.5, label="model"):
    auc  = roc_auc_score(y_true, y_pred_proba)
    brier = brier_score_loss(y_true, y_pred_proba)
    y_pred = (y_pred_proba >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    return {"label": label, "AUC": auc, "Brier": brier, "Accuracy@0.5": acc}

res = []
res.append(eval_metrics(y_test, logit_proba, label="Logistic Regression"))
res.append(eval_metrics(y_test, gb_proba, label="Gradient Boosting"))
pd.DataFrame(res)


## 9. Feature importance and interpretation

In [ ]:
# Logistic Regression coefficients (top magnitude)
coef_series = pd.Series(logit.coef_[0], index=X_train.columns)
top_coef = coef_series.reindex(coef_series.abs().sort_values(ascending=False).head(20).index)

plt.figure()
top_coef.sort_values().plot(kind="barh")
plt.title("Feature importance vs Log-odds (Logistic Regression)")
plt.xlabel("Coefficient")
plt.tight_layout()
plt.show()

# Permutation importance for Gradient Boosting
perm = permutation_importance(gb, X_test, y_test, n_repeats=10, random_state=42)
perm_imp = pd.Series(perm.importances_mean, index=X_train.columns).sort_values(ascending=False).head(20)

plt.figure()
perm_imp.sort_values().plot(kind="barh")
plt.title("Permutation importance (Gradient Boosting)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()


## 10. Predict win probabilities for a given week

In [ ]:
# Filter the target week in PREDICT_SEASON and PREDICT_WEEK
target_games = games_valid[(games_valid["season"] == PREDICT_SEASON) & (games_valid["week"] == PREDICT_WEEK)].copy()
target_X = target_games[X.columns].copy()

# Scale for logistic
target_X_scaled = scaler.transform(target_X)

target_games["proba_home_logit"] = logit.predict_proba(target_X_scaled)[:,1]
target_games["proba_home_gb"] = gb.predict_proba(target_X)[:,1]
target_games["pick_logit_home"] = (target_games["proba_home_logit"] >= 0.5).astype(int)
target_games["pick_gb_home"] = (target_games["proba_home_gb"] >= 0.5).astype(int)

cols_show = ["season","week","home_team","away_team","home_win","proba_home_logit","proba_home_gb","pick_logit_home","pick_gb_home"]
target_games[cols_show].sort_values(["week","home_team"]).reset_index(drop=True)


## 11. Calibration and comparison

In [ ]:
# Scatter comparison of model probabilities (independent vs. dependent order: Logistic vs GradientBoosting)
plt.figure()
plt.scatter(logit_proba, gb_proba, alpha=0.6)
plt.xlabel("Logistic Regression predicted probability")
plt.ylabel("Gradient Boosting predicted probability")
plt.title("Logistic Regression vs Gradient Boosting predicted probabilities")
plt.tight_layout()
plt.show()


## 12. Notes and caveats

- Early weeks can have sparse lag features.  Consider requiring at least three prior weeks or backfilling with priors.  
- RBSDM stats are team-season level here.  You can extend to team-week by scraping their weekly tables or computing directly from pbp.  
- Try alternative models like XGBoost or LightGBM if you want stronger non-linear learners.  
- Consider adding market-based priors (closing spreads, totals) to improve calibration.  
